In [1]:
# Dependencies
import matplotlib.pyplot as plt
from citipy import citipy as cp
import pandas as pd
import requests as req
import json
import time

In [ ]:
#from citipy, retrieve list of cities based on lat and lng 
list_of_cities = []
counter = 0
Duplicates  = 'no'
while counter <= 550:
    for x in range(90,-90,1):
        for y in range(180,-180,1):
            if counter > 550: continue   
            city = cp.nearest_city(x, y)
            citdict = {}
            citdict['city'] = city.city_name
            citdict['country'] = city.country_code
            citdict['lat'] = x
            citdict['long'] = y
            if len(list_of_cities) == 0:
                list_of_cities.append(citdict)
                counter+=1
                continue
            else:
            #Get rid of duplicates
                for city in list_of_cities:
                    if city['city'] == citdict['city']:
                        Duplicates = 'yes'
            if Duplicates == 'no':
                list_of_cities.append(citdict)
                counter+=1
            else:
                Duplicates = 'no'

print(len(list_of_cities))  

In [ ]:
print(list_of_cities[0])

In [ ]:
#Creating dataframe of 549 cities
cities_df = pd.DataFrame({
    'city': [x['city'] for x in list_of_cities],
    'country': [x['country'] for x in list_of_cities],
})

In [ ]:
cities_sample = cities_df.sample(549)
cities_sample

In [ ]:
url = "http://api.openweathermap.org/data/2.5/weather?"
api_key = '745e3de348c09bb314ceec9ad93787f5'

In [ ]:
units = "Imperial"
count = 0
cities_sample['latitude'] = ""
cities_sample['longitude'] = ""
cities_sample['temperature'] = ""
cities_sample['humidity'] = ""
cities_sample['cloudiness'] = ""
cities_sample['wind_speed'] = ""

for index,row in cities_sample.iterrows():
    count+= 1
    query_url = url + "appid=" + api_key + "&units=" + units + "&q=" + row['city']
    try:
        weather_response = req.get(query_url)
        cityweather = weather_response.json()
#         print(cityweather)
        cities_sample.set_value(index, "latitude", int(cityweather['coord']['lat']))
        cities_sample.set_value(index, "longitude", int(cityweather['coord']['lat']))
        cities_sample.set_value(index, "temperature", int(cityweather['main']['temp']))
        cities_sample.set_value(index, "humidity", int(cityweather['main']['humidity']))
        cities_sample.set_value(index, "cloudiness", int(cityweather['clouds']['all']))
        cities_sample.set_value(index, "wind_speed", int(cityweather['wind']['speed']))
    except:
        print(f"No data for this city: {row['city']}")
    print(f"This is city#: {count}")
    print(f"This is: {row['city']}" )
    print(f"This is the requested URL: {query_url}")

In [ ]:
cities_sample = cities_sample[cities_sample.latitude != ""]

In [ ]:
cities_sample

In [ ]:
# to test the accuracy of sampled cities distribution 
plt.scatter(cities_sample['latitude'],cities_sample['longitude'])
plt.title(f"Sample Distribution ")
plt.xlabel("Latitude")
plt.ylabel("longitude")
plt.show()

In [ ]:
#using today's date
date = time.strftime("%m/%d/%Y")
# ploting Temp and lat
plt.scatter(cities_sample['latitude'],cities_sample['temperature'])
plt.title(f" City Latitude Vs. Temperature {date}")
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.style.use('ggplot')
plt.savefig("Temperature.png")

In [ ]:
plt.show()

In [ ]:
# plting lat and humidity
plt.scatter(cities_sample['latitude'], cities_sample['humidity'])
plt.title(f"City Latitude Vs. Humidity {date}")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.style.use('ggplot')
plt.savefig("Humidity.png")

In [ ]:
plt.show()

In [ ]:
# plting lat and cloudiness 
plt.scatter(cities_sample['latitude'], cities_sample['cloudiness'])
plt.title(f"City Latitude Vs. Cloudiness {date}")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.style.use('ggplot')
plt.savefig("Cloudiness.png")

In [ ]:
plt.show()

In [ ]:
# plting lat and windspeed
plt.scatter(cities_sample['latitude'], cities_sample['wind_speed'])
plt.title(f"City Latitude Vs. Wend Speed {date}")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed(mph)")
plt.style.use('ggplot')
plt.savefig("Wind_Speed.png")

In [ ]:
plt.show()